<a href="https://colab.research.google.com/github/mohammad-rahbari/Federated-Learning-MLDL/blob/master/notebooks/FederatedLearning_Integrated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Importing DINO and installing its dependencies

In [2]:
# @title Clon the DINO ripo
!git clone https://github.com/facebookresearch/dino.git

Cloning into 'dino'...
remote: Enumerating objects: 175, done.
remote: Total 175 (delta 0), reused 0 (delta 0), pack-reused 175 (from 1)
Receiving objects: 100% (175/175), 24.47 MiB | 13.01 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [3]:
# @title Installing required dependencies regarding DINO
%cd dino
!pip install -r requirements.txt
!pip install timm

/content/dino
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split,DataLoader


# preprocessing the CIFAR-100 dataset

feature size in CIFAR is 32x32 but DINO requires 224x224 in the input layer.

In first step we upscale the dataset and then we add randomization to it

In last step of transformation we normalize data usind mean value and standard division of ImageNet



In [5]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])

In [6]:
from torch.utils.data import ConcatDataset
import numpy as np
from torchvision.datasets import CIFAR100
train_dataset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
full_train = train_dataset
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=2)
# Verify the length of the new dataset
print(f"Length of combined dataset: {len(full_train)}")


100%|██████████| 169M/169M [00:12<00:00, 13.0MB/s]


Length of combined dataset: 50000


In [7]:

#@title Imports
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset
import numpy as np
import random
import torch
import pandas as pd
import os
import math

# Set Hyperparameters regarding the data spliting here!

In [8]:

#@title set the parameters here!!



number_of_clients = None
train_frac = 1 #@param
val_frac = 0 #@param
batch_size = 32 #@param{type:"integer"}
is_seed_fixed = True #@param{type:"boolean"}
seed = 42 #@param{type:"integer"}

def set_seed(seed=42, is_seed_fixed=True):
  if not is_seed_fixed:
    return
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False


set_seed(seed,is_seed_fixed)



#@markdown </br> <h5>Indicate the number of clients that contribute in training:</h5>
n_clients = 100 #@param{type:"integer"}

#@markdown </br></br> <b>splitting hyperparameters</b>

spliting_method = "non-i.i.d. sharing" #@param["i.i.d. sharing","non-i.i.d. sharing"]
backbone = "dino_vits16" #@param["dino_resnet50", "dino_vits16", "dino_xcit_small_12_p16"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [65]:
#@title Set the parameters here only if <b>non-i.i.d. sharing</b> method had been selected!!
#@markdown Nc is the number of classes that each subset can contain
if spliting_method == "non-i.i.d. sharing":
  Nc = 10 #@param{type:"integer"}

  # are_classes_overlaping = False #@param{type:"boolean"}

#@markdown <h3>If we consider the Number of classes M and nummber of client K then:</h3>
#@markdown <ul>
#@markdown   <li>Nc should be:
#@markdown     <ul>
#@markdown       <li>
#@markdown         Greater than or equal to <b>\\(\frac{M}{K}\\)</b>
#@markdown       </li>
#@markdown       <li>
#@markdown         Less than or equal to K </b>
#@markdown       </li>
#@markdown     </ul>
#@markdown   </li>
#@markdown   <li>
#@markdown   Muximum number of clients means all classes contribute in every client
#@markdown   </li>

#@markdown </ul>


#@markdown </br></br><h3>Combination of classes are randomly selected which suits definition of federated learning especially Cross-device federated learning</h3>





# Data splitting

In [11]:
# @title data splitting

set_seed(seed,is_seed_fixed)
generator = torch.Generator().manual_seed(seed)

total_size = len(full_train)
train_size = int(train_frac * total_size)
val_size   = total_size - train_size

train_set, val_set = random_split(full_train, [train_size, val_size], generator=generator)
train_indices = torch.tensor(train_set.indices)
val_indices = torch.tensor(val_set.indices)

train_set = Subset(train_set.dataset, train_indices)
val_set = Subset(val_set.dataset, val_set.indices)

train_loader = DataLoader(train_set, batch_size=len(train_set), shuffle=False)
if val_size > 0:
  val_loader  =  DataLoader(val_set, batch_size=len(val_set), shuffle=False)
  print(f"Validation dataset size: {len(val_set)}")




print(f"Train dataset size: {len(train_set)}")

lenghts = [train_size//n_clients] * n_clients

for i in range(train_size % n_clients):
  lenghts[i] += 1
print("Size of subset: ", lenghts)





Train dataset size: 50000
Size of subset:  [500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500]


In [12]:
# @title i.i.d sharing - split data dased on number of clients and with respect of label proportionality
set_seed(seed,is_seed_fixed)
def iid_sharing(dataset, n_clients):

  full_train_indices = dataset.indices
  full_train_labels = torch.from_numpy(np.array(dataset.dataset.targets)[full_train_indices]) #collects labels from all dataset
  unique_lables = torch.unique(full_train_labels) #Removes dupilication and generates a uniuqe list of labels (classes)
  classes_indices = {}


  for i in unique_lables:
    classes_indices[i] = (full_train_labels == i).nonzero(as_tuple=True)[0] #Collects and save Indices in an array based on classes

  for i in classes_indices.keys():
    classes_indices[i] = classes_indices[i][torch.randperm(classes_indices[i].shape[0])] #suffels the indices


  client_indices = {}



  for client in range(n_clients):
    if not client_indices.get(client):
      client_indices[client] = torch.empty(0, dtype=torch.long)


  for label in classes_indices.keys():
    class_sample_size = len(classes_indices[label]) // n_clients
    for k in range(n_clients):

      client_indices[k] = torch.cat((client_indices[k], classes_indices[label][class_sample_size * k : class_sample_size * (k + 1)]), dim=0)

  for label in classes_indices.keys():
    remainder  = len(classes_indices[label]) % n_clients

    for r in range(1,remainder+1):
      random_client = random.choice(list(client_indices.keys()))
      client_indices[random_client] = torch.cat((client_indices[random_client], classes_indices[label][-r].unsqueeze(0)), dim=0)

  full_train_indices_t = torch.as_tensor(full_train_indices, dtype=torch.long)
  client_data = {
      client_id: Subset(
          dataset.dataset,
          full_train_indices_t[indices][torch.randperm(len(indices))].tolist()
      )
      for client_id, indices in client_indices.items()
  }


  #split actual dataset to multiple subset for clients
  # client_data={
  #     client_id: Subset(dataset.dataset,indices[torch.randperm(len(indices))])
  #     for client_id, indices in client_indices.items()
  # }
  return client_data

# indices_check = []
# client_data = iid_sharing(train_set, n_clients)
# s = 0
# for client_id in client_data.keys():
#   indices_check = indices_check + list(client_data[client_id].indices)
#   s+= len(client_data[client_id])
#   print(f"Client {client_id} has {len(client_data[client_id])} samples")
# print(s, len(train_set))
# del indices_check,client_data

In [13]:
# @title Non i.i.d sharing


# @title i.i.d sharing - split data dased on number of clients and with respect of label proportionality
set_seed(seed,is_seed_fixed)
def noniid_sharing(dataset,Nc , n_clients):

  full_train_indices = dataset.indices
  full_train_labels = torch.tensor(dataset.dataset.targets)[full_train_indices] #collects labels from all dataset
  unique_labels = torch.unique(full_train_labels) #Removes dupilication and generates a uniuqe list of labels (classes)

  label_to_pos = {int(l.item()): idx for idx, l in enumerate(unique_labels)}

  classes_indices = {}
  classes_size = torch.zeros(unique_labels.size()[0])


  class_combs = get_class_combinations(unique_labels, Nc, n_clients)

  classes_num_partition = torch.zeros(unique_labels.size()[0])

  for i in unique_labels:
    classes_num_partition[label_to_pos[int(i.item())]] = torch.sum(class_combs == i)

    subset_pos = (full_train_labels == i).nonzero(as_tuple=True)[0]
    abs_idx = torch.as_tensor(full_train_indices, dtype=torch.long)[subset_pos]

    classes_indices[i.item()] = abs_idx  #Collects and save Indices in an array based on classe

    classes_size[label_to_pos[int(i.item())]] = abs_idx.size(0) #Calculate the number of smaples belonging to each class

  for i in classes_indices.keys():
    classes_indices[i] = classes_indices[i][torch.randperm(classes_indices[i].shape[0])] #suffels the indices

  client_indices = {client: torch.tensor([],dtype=torch.int64) for client in range(n_clients) }
  assigned_indices = set()

  #For each client we generate a element in client_indices dict to keep track of indices we'll associated with each client

  for client in range(n_clients):
    for cls in class_combs[client]:
      cls = cls.item()

      pos = label_to_pos[int(cls)]
      portion  = classes_size[pos] /classes_num_partition[pos]
      portion = int(portion) if not portion % 1 else int(portion) + 1
      portion = min(portion, classes_indices[cls].size()[0])

      class_partition = classes_indices[cls][:portion]

      class_partition_list = class_partition.tolist()
      class_partition_list = [idx for idx in class_partition_list if idx not in assigned_indices]

      assigned_indices.update(class_partition_list)

      class_partition = torch.tensor(class_partition_list, dtype=torch.int64)

      client_indices[client] = torch.cat((client_indices[client], class_partition), dim=0)

      classes_indices[cls] = classes_indices[cls][portion:]

  client_data={
      client_id: Subset(dataset.dataset, indices[torch.randperm(len(indices))])
      for client_id, indices in client_indices.items()
      if len(indices) > 0
  }



  return client_data, class_combs





def get_class_combinations(classes, Nc, n_clients):

  if Nc * n_clients < len(classes):
    Nc = len(classes) / n_clients
    Nc = int(Nc) if not Nc % 1 else int(Nc) + 1

    print(f"Number of classes per clients is lower then minimum. Nc changed to {Nc} (the least possible value)")

  combinations = torch.zeros((n_clients,Nc),dtype= torch.int64)
  counter =0
  ofset = 0
  flag = False

  for i in range(n_clients):
    if not flag:
      end_pointer = (i + 1) * Nc
      if end_pointer >= classes.size()[0]:
          ofset = (end_pointer - classes.size()[0])
          flag = True

      combinations[i] = classes[i* Nc - ofset: end_pointer - ofset]

    else:

      combinations[i]  = torch.randperm(classes.size()[0])[:Nc]

  return combinations
# client_data, class_combs = noniid_sharing(train_set,Nc=Nc, n_clients=n_clients)

# print(class_combs[1])
# temp=DataLoader(client_data[2], batch_size=len(client_data[2]))
# for image , lbl in temp:

#   print(len(torch.unique(lbl)))


# Log System

In this section Requerd Data will be stored.<br/><br/>
**Archaving this information will make it possible to:**
*   Handle Clients
*   Manage the models
*   Keep track of results of different Backbones
*   Compare measurement criteria
*   Handel model merging process
*   Save path to the models

<br/><br/>
**These data will be saved in two seperted csv file to :**

1.   Store the LOCAL Models  
2.   Store the GLOBAL Models resulted by each round

<br/><br/>
The csv files will be handeled as panda.dataframe and each row in the csv file addresses one of models
<br/>

**Columns (COMMON):**<br/>
1. Backbone model name
2. Model name
3. Path
4. Time of log
5. Measurement criteria
 * loss
 * Accuracy
 * ...?
6. Size of dataset

**Columns (Local Models only):**<br/>
7. Client Id
8. Classes (Indicate which classes have been covered by each client)(format:"2,4,63,80,9" or "all" for all the classes)
9. Round number
10. Duration of training
11. Train Test ratio

**Columns (Global Models only):**<br/>
7. Number of clients
7. Number of rounds
8. Model Aggregation method







In [14]:
# @title Functions
import torch
from datetime import datetime
import time
from google.colab import drive
from uuid import uuid4
import os

def get_current_time():
  now = datetime.now()

  formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S") # Format the date and time as a string

  return formatted_date_time



tic_start_time = None

def next_id(log_path):
  if os.path.exists(log_path) :
    df = pd.read_csv(log_path)
    while True:
      uuid = str(uuid4())
      if uuid not in df["model_name"].values:
        return uuid
  else:
    return str(uuid4())



def tic():
    global tic_start_time
    tic_start_time = time.perf_counter() # start the timer

def toc():
    if tic_start_time is None:
        print("Error: You must call tic() before toc()")
        return None
    elapsed_time = time.perf_counter() - tic_start_time
    return elapsed_time




# Model and model configuration

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict

class DinoClassifier(nn.Module):
  def __init__(self, dino_model, num_classes:int=100, device=None):
    super(DinoClassifier, self).__init__()
    self.backbone = dino_model

    #We need to freaze thhe parameters of bakbone first so we can train only on the head layer(output layer)
    for param in self.backbone.parameters():
      param.requires_grad = False

    #determine the Device
    if device is None:
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    self.backbone.to(device)

    #To detect the output feature dimontion of backbone we run  Dummy forward pass
    with torch.no_grad():

      dummy_input = torch.randn(1,3,224,224).to(device)
      dummy_out = self.backbone(dummy_input)


      if isinstance(dummy_out, tuple):
        dummy_out = dummy_out[0]
      elif isinstance(dummy_out, dict):
        dummy_out = dummy_out.get("x_norm_clstoken", next(iter(dummy_out.values())))

      #If the output is 3D (B, T, D), we assume first token is the [CLS] token.
      if dummy_out.dim() == 3:
        dummy_feature = dummy_out[:,0]
      else:
        dummy_feature = dummy_out
      feature_dim = dummy_feature.shape[1]

      #Difineing the classification Head
      self.head = nn.Linear(feature_dim, num_classes)

      #Ensure the head is trainable.
      for param in self.head.parameters():
        param.requires_grad = True

  def forward(self,x):

    #pass the input through the backbone
    features = self.backbone(x)

    if isinstance(features, tuple):
      features = features[0]
    elif isinstance(features, dict):
      features = features.get("x_norm_clstoken", next(iter(features.values())))


    # If featers are retuened as (B, T, D), use the first token
    if features.dim() == 3:
      cls_token = features[:,0]
    else:
      cls_token = features
    logits = self.head(cls_token)

    return logits




# Clients

In [20]:
from torch.utils.data import random_split
from torch.utils.data import Subset
import pandas as pd
import torch.hub
import copy
from collections import defaultdict
from torch.utils.data import DataLoader # Import DataLoader

set_seed(seed,is_seed_fixed)

class Client:


  def __init__(self, id, data, n_clients, spliting_method,num_local_steps = 4,GM_rounds=5, target_sparsity=0.9, batch_size = 32, classes="all",  backbone="dino_vits16", path_to_model=None, raw_model=None,do_test=False, spliting_ratio={"train":0.8, "test":0.2}, path_to_subsets="", path_to_class_combs=""):
    self.id = id
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Corrected cuda() to is_available()
    self.data_set = data
    self.spliting_method = spliting_method
    self.classes = classes

    self.backbone = backbone
    self.path_to_model = path_to_model
    self.model = raw_model
    self.load_model()
    self.do_test = do_test
    self.batch_size = batch_size

    self.train_set , self.test_set = self.test_train_split()
    self.target_sparsity = target_sparsity

    self.GM_rounds = GM_rounds
    self.sparsities = self.claculate_sparsity()
    self.grad_mask, self.fisher_score_matrix = self.calculate_fisher_mask()
    self.n_clients = n_clients


    self.spliting_ratio = spliting_ratio

    self.num_local_steps = num_local_steps
    self.selected_batches = []
    self.get_random_batches()
    self.duration = 0.0
    self.train_loss = None
    self.accuracy = None
    self.loss = None
    self.path_to_subsets = path_to_subsets
    self.path_to_class_combs = path_to_class_combs


  def test_train_split(self):
    if self.do_test:
      train_size = int(self.spliting_ratio.get("train") * len(self.data_set))
      test_size =  len(self.data_set) - train_size

      train_set, test_set = random_split(self.data_set, [ train_size, test_size ])
      train_set = DataLoader(train_set, batch_size=self.batch_size, shuffle=True,  num_workers=2)
      test_set = DataLoader(test_set, batch_size=self.batch_size, shuffle=False,  num_workers=2)
      return train_set, test_set

    else:
      spliting_ratio={"train":1, "test":0}
      train_set = DataLoader(self.data_set, batch_size=self.batch_size, shuffle=True,  num_workers=2)
      return train_set , None

  def get_random_batches(self):
    total_batches = len(self.train_set)

    selected_indices = torch.sort(torch.randperm(total_batches)[:self.num_local_steps])[0]

    selected_indices = list(set(selected_indices.tolist()))
    random.shuffle(selected_indices)
    self.selected_batches = []

    for i, batch in enumerate(self.train_set):
      if i in selected_indices:

        self.selected_batches.append(batch)
    # print("Total number of batches: ",total_batches, " - Number of selected batches: ", len(self.selected_batches), "selected batches: ",selected_indices )


  def load_model(self):
    if self.model is None:
      dino_model = torch.hub.load('facebookresearch/dino:main', self.backbone)
      self.model = DinoClassifier(dino_model=dino_model, num_classes=100, device=self.device)
    if self.path_to_model:
      state_dict = torch.load(self.path_to_model)
      self.model.head.load_state_dict(state_dict)
      # print("Model loaded successfully ")

    self.model.to(self.device)

  def gradient_mask(self):


    for name, param in self.model.head.named_parameters():
        if name in self.grad_mask and param.grad is not None:

          param.grad *= self.grad_mask[name].to(param.grad.device)



  def calculate_fisher_mask(self, n=5,last_mask=None ):
    criterion = nn.CrossEntropyLoss()

    head_params = [p for p in self.model.head.parameters()]
    param_ids = [id(p) for p in head_params]
    fisher_scores = {id_p:torch.zeros_like(p, device=self.device) for id_p,p in  zip(param_ids, head_params)}

    last_mask = {id_p:torch.ones_like(p, device=self.device) for id_p,p in  zip(param_ids, head_params)}



    self.model.eval()

    for i in range(n):

      for v in fisher_scores.values():
        v.zero_()

      for images, labels in self.train_set:
        images, labels = images.to(self.device), labels.to(self.device)
        outputs = self.model(images)
        loss = criterion(outputs, labels)

        grads = torch.autograd.grad(
            loss,
            head_params,
            create_graph=False,
            retain_graph=False
        )

        for p, g in zip(head_params,grads):
          pid = id(p)
          fisher_scores[pid] += g.detach().pow(2) * last_mask[pid]

      all_scores = torch.cat([
           fisher_scores[id(p)].mul(last_mask[id(p)]).reshape(-1)
          for p in  head_params
          ])

      non_zero  = all_scores[all_scores != 0]

      if non_zero.numel() == 0:
        new_mask = {id(p):torch.zeros_like(p,device=self.device) for p in head_params }
        last_mask = new_mask
        continue

      total_nz = non_zero.numel()
      keep = int( (1 - self.sparsities[i])* total_nz)
      keep = min(keep, total_nz)

      if keep == 0:
        threshold = non_zero.max() + 1
      elif keep == total_nz:
        threshold = non_zero.min() - 1

      else:
        kth_smallest = total_nz - keep + 1
        threshold, _ = torch.kthvalue(non_zero, k= kth_smallest)

      new_mask = {}

      for p in head_params:
        pid = id(p)
        masked_scores = fisher_scores[pid] * last_mask[pid]
        current_mask = (masked_scores >= threshold).float() * last_mask[pid]
        new_mask[pid]  = current_mask
        last_mask[pid] = current_mask

    return new_mask, fisher_scores

  def set_final_grad_mask(self, global_mask):
    spec = []
    for name, p in initial_model.head.named_parameters():
    spec.append({
        "name": name,
        "shape": tuple(p.shape),
        "numel": p.numel()
    })

    head_params = [p for p in self.model.head.parameters()]
    for p in head_params:
        pid = id(p)
        self.grad_mask[pid] = self.grad_mask[pid] * global_mask[pid]


  def claculate_sparsity(self, a= 0.4, decimals= 3):
    n =  self.GM_rounds
    S = self.target_sparsity
    St = []
    Ct = [ S * (  (1-math.exp(-a*t)) / (1-math.exp(-a*n))  ) for t in  range(1,n+1) ]
    for t in  range(n):
      ct0 = 0 if t==0 else Ct[t-1]
      St.append( round(1- (1-Ct[t])/(1-ct0), decimals ) )
    return St


  @torch.no_grad()
  def SGDM(self, buffer,weight_decay=0.0,lr=1e-3,momentum=0.9,damping=0.0, nesterov=False, max_=False):
    for param in self.model.head.parameters():
      if param.grad is None:
        continue
      grad = param.grad

      if weight_decay != 0:
        grad = grad.add(param, alpha=weight_decay)

      pid = id(param)
      buf = buffer.get(pid)
      if buf is None:
        buf = torch.zeros_like(param)
        buffer[pid] = buf


      if momentum != 0 :
        buf.mul_(momentum).add_(grad, alpha=(1 - damping))
        update = grad.add(buf, alpha= momentum) if nesterov else buf

      else:

        update = grad

      gm = self.grad_mask
      mask = gm.get(pid)

      if mask is None:
        mask = torch.ones_like(update)
      else:
        mask = mask.to(device=update.device, dtype=update.dtype)
        if mask.numel() == 1:
          mask = mask.expand_as(update)
        elif mask.shape != update.shape:

          mask = mask.expand_as(update)

      update = update * mask

      param.add_(update, alpha=(lr if max_ else -lr))

      buffer[pid] = buf
    return buffer









  def SGDM_train(self):
    criterion = nn.CrossEntropyLoss()

    tic()



    for p in self.model.backbone.parameters():
      p.requires_grad = False
    self.model.backbone.eval()

    for p in self.model.head.parameters():
        p.requires_grad = True

    self.model.train()

    buffer = {}
    running_loss = 0.0
    loss_lst = {}


    for step_num in range(len(self.selected_batches)):

      images, labels = self.selected_batches[step_num]
      images, labels = images.to(self.device), labels.to(self.device)

      self.model.zero_grad(set_to_none=True)
      outputs = self.model(images)
      loss = criterion(outputs, labels.long())

      loss.backward()

      buffer = self.SGDM(buffer=buffer, momentum=0.9)





      running_loss += loss.item()
      # loc_stp_loss = running_loss / len(self.train_set)
      avg_loss = running_loss / (step_num + 1)
      loss_lst[step_num] = f"{avg_loss:.4f} "

      # print(f"client: {self.id}- local step number: {step_num} - step loss: {avg_loss:.4f} " )
    print(f"client:{self.id}_  losses",loss_lst)

    self.duration = toc()
    self.train_loss = running_loss / len(self.selected_batches)
    # self.train_loss = running_loss / max(1, len(self.selected_batches))

  def train_default(self):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(self.model.head.parameters(), lr=1e-3, momentum=0.9)
    tic()
    self.calculate_fisher_mask()


    for p in self.model.backbone.parameters():
      p.requires_grad = False
    self.model.backbone.eval()

    for p in self.model.head.parameters():
        p.requires_grad = True

    self.model.train()

    running_loss = 0.0
    loss_lst = {}
    for step_num in range(len(self.selected_batches)):


      images, labels = self.selected_batches[step_num]
      images, labels = images.to(self.device), labels.to(self.device)

      optimizer.zero_grad(set_to_none=True)
      outputs = self.model(images)
      loss = criterion(outputs, labels.long())
      loss.backward()


      for p in self.model.head.parameters():
            if p.grad is None:
                continue
            m = self.grad_mask.get(id(p))
            if m is not None:
                p.grad.mul_(m.to(p.grad.device, dtype=p.grad.dtype))

      optimizer.step()

      running_loss += loss.item()

      # loc_stp_loss = running_loss / len(self.train_set)
      avg_loss = running_loss / (step_num + 1)
      loss_lst[step_num] = f"{avg_loss:.4f} "

    print(f"client:{self.id}_  losses",loss_lst)
    self.duration = toc()
    self.train_loss = running_loss / len(self.selected_batches)
    # self.train_loss = running_loss / max(1, len(self.selected_batches))

  def evaluate(self):
    if self.do_test:
      criterion = nn.CrossEntropyLoss()
      self.model.eval()
      correct = 0
      total = 0
      test_loss = 0
      with torch.no_grad():
        for index, (images, labels) in enumerate(self.train_set):
          images, labels = images.to(self.device), labels.to(self.device)
          outputs = self.model(images)

          _, prediction = torch.max(outputs.data,1)
          loss = criterion(outputs, labels)
          test_loss += loss.item() * labels.size(0)


          total += labels.size(0)

          correct += (prediction == labels).sum().item()
      self.accuracy = 100 * correct / total
      self.loss = test_loss / total
    else:
      self.accuracy = None
      self.loss = None

  def confirm_save(self,path):
      torch.save(self.model.head.state_dict(),  path )

  def create_log(self, model_name, path, round_number):

    log_dict= {
        "client_id":[self.id],
        "backbone":[self.backbone],
        "model_name":[model_name],
        "initial_model_name":[initial_model_name],
        "path": [path],
        "num_of_clients":[self.n_clients],
        "Measurement_criteria":["accuracy,loss,train_loss" if self.do_test else "train_loss"],
        "accuracy":[self.accuracy],
        "loss":[self.loss],
        "train_loss":[self.train_loss],
        "splitting_method":[self.spliting_method],
        "sparsity":[self.target_sparsity],
        "sparsities":[self.sparsities ] ,
        "GM_rounds":[self.GM_rounds],
        "num_local_steps":[self.num_local_steps],
        "size_of_dataset": [len(self.data_set.dataset)],
        "client_train_size": [len(self.train_set.dataset)],
        "client_test_size": [len(self.test_set.dataset) if self.do_test else 0],
        "train_test_ratio":[self.spliting_ratio],
        "classes":[self.classes],
        "round_number":[round_number],
        "duration":[self.duration],
        "time": [get_current_time()],
        "path_to_subsets":[self.path_to_subsets],
        "path_to_class_combs":[self.path_to_class_combs]

    }
    final_log = pd.DataFrame(log_dict)
    final_log = final_log[['client_id', 'backbone', 'model_name', 'initial_model_name', 'path','num_of_clients','train_loss', 'splitting_method', 'sparsity', 'sparsities', 'GM_rounds','size_of_dataset', 'client_train_size', 'client_test_size', 'train_test_ratio', 'classes', 'round_number','num_local_steps', 'duration', 'time','path_to_subsets', 'path_to_class_combs']]

    return final_log

In [21]:
# temp_clients_log = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv")
# temp_clients_log["backbone"]= ["dino_vits16"]*len(temp_clients_log)
# temp_clients_log["path_to_subsets"]= ["/content/drive/MyDrive/MLDL_FederatedLearning/client_subsets/client_data_iid_100clients_5455920f-4657-43ce-8e9e-accab67d8bfd.pth"]*len(temp_clients_log)
# temp_clients_log["path_to_class_combs"]= [None]*len(temp_clients_log)
# temp_clients_log["num_local_steps"] = [5]*len(temp_clients_log)
# temp_clients_log = temp_clients_log[['client_id', 'backbone', 'model_name', 'initial_model_name', 'path',
#        'num_of_clients',
#        'train_loss', 'splitting_method', 'sparsity', 'sparsities', 'GM_rounds',
#        'size_of_dataset', 'client_train_size', 'client_test_size',
#        'train_test_ratio', 'classes', 'round_number','num_local_steps', 'duration', 'time',
#        'path_to_subsets', 'path_to_class_combs']]
# temp_clients_log.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv", index=False)

# temp_clients_log[-10:].head()

# Training loop

**<h1>❗ Important Notice ❗</h1>**

**Regarding `save_data`:**
Please be aware that checking the `save_data` option will generate a **new data subset** and a **new initial model** based on your specified parameters.

**⚠️ Crucial: Using Existing Models with New Data/Parameters ⚠️**
If you intend to use an *existing model* but wish to apply it to a *different data subset*, use a *different data splitting method*, or make *any other changes to the data or algorithm*, you **MUST** assign a **new and unique model name**.

**Why is this critical?**
Failing to use a unique model name will make it impossible to differentiate between models for each client when filtering. This will lead to inaccurate results from the client aggregation function on the server.

In [66]:
from uuid import uuid4
save_data = True #@param{"type":"boolean"}

if save_data:
  method = "iid" if spliting_method == "i.i.d. sharing" else "noniid"
  path_to_subsets = f"/content/drive/MyDrive/MLDL_FederatedLearning/client_subsets/client_data_{method}_{str(n_clients)}clients_{str(uuid4())}.pth"
  print(method)
  if spliting_method == "i.i.d. sharing":
    path_to_class_combs = None
    client_data = iid_sharing(train_set, n_clients)
    class_combs = "all"
    print(spliting_method)
  else:

    client_data, class_combs = noniid_sharing(train_set,Nc=Nc, n_clients=n_clients)
    path_to_class_combs = f"/content/drive/MyDrive/MLDL_FederatedLearning/client_subsets/class_combs_{method}_{str(n_clients)}clients_{str(uuid4())}.pth"
    print(spliting_method)
    print("Nc: ",Nc)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  dino_model = torch.hub.load('facebookresearch/dino:main', backbone)
  initial_model = DinoClassifier(dino_model=dino_model, num_classes=100, device=device)
  initial_model_name = next_id("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
  initial_model_path = "/content/drive/MyDrive/MLDL_FederatedLearning/models/global/" + initial_model_name + ".pth"
  initial_model_round_num = 0
  initial_model_log = {
    "backbone": [backbone],
    "model_name": [initial_model_name],
    "num_of_clients": [n_clients],
    "path": [initial_model_path],
    "Measurement_criteria": [None],
    "prev_global_model_name":[None],
    "accuracy": [None],
    "loss": [None],
    "splitting_method": [spliting_method],
    "size_of_dataset": [len(train_dataset)],
    "train_test_ratio": [None],
    "round_number": [0],
    "time": [get_current_time()],
    "path_to_subsets": [path_to_subsets],
    "path_to_class_combs": [path_to_class_combs],
    "num_of_participants": [None],
    "Nc": [None],
    "num_local_steps": [None]


}


  initial_model_log["aggregation_method"] =[ np.nan]
  initial_model_log["contributors"] =[ np.nan]
  initial_model_log["momentum_vector_path"] = [np.nan]


  initial_model_log = pd.DataFrame(initial_model_log)
  initial_model_log = initial_model_log[
      ['backbone',
        'num_of_clients',
        'splitting_method',
        'Nc',
        'num_local_steps',
        'aggregation_method',
        'Measurement_criteria',
        'accuracy',
        'loss',
        'size_of_dataset',
        'train_test_ratio',
        'round_number',
        'num_of_participants',
        'model_name',
        'prev_global_model_name',
        'contributors',
        'path',
        'momentum_vector_path',
        'path_to_subsets',
        'path_to_class_combs',
        'time']
      ]

  if not os.path.exists("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv"):
    initial_model_log.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv", index=False)
  else:
    initial_model_log.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv", mode='a', header=False, index=False)
    print("New record")
  torch.save(initial_model.head.state_dict(), initial_model_path)

  torch.save(client_data, path_to_subsets)
  if method== "noniid":
    torch.save(class_combs, path_to_class_combs)

  print(  initial_model_log['model_name'])




noniid
non-i.i.d. sharing
Nc:  10


Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


New record
0    4b94d332-b21c-4e2b-af6f-f7801a35fc4c
Name: model_name, dtype: object


In [24]:
# @title Gets the last generation
def get_lastgen_model_name(seed_model_name):
  chain = ""
  while True:

    chain += f"{seed_model_name} -->"

    global_log_df = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
    prev_models_names = global_log_df["prev_global_model_name"].values

    if seed_model_name in prev_models_names:
      log_buffer = global_log_df[global_log_df["prev_global_model_name"] == seed_model_name ]
      seed_model_name = log_buffer["model_name"].values[0]

    else:
     print(chain)
     return seed_model_name


In [25]:
# @title Set clients parameters

local_steps = 8 #@param
sparcity = 0.5 #@param
fisher_information_matrix_colibration_rounds = 5 #@param


In [55]:
# @title Client round
set_seed(seed,is_seed_fixed)

def flatten_from_pid_dict_using_names(score_dict, client_model, spec_by_name):

  name2pid = {name: id(p) for name, p in client_model.head.named_parameters()}

  return torch.cat([score_dict[name2pid[s["name"]]].reshape(-1) for s in spec_by_name])
def unflatten_to_named(vec, spec_by_name, device=None, dtype=None):
  out, off = {}, 0
  for s in spec_by_name:
    n = s["numel"]
    t = vec[off:off+n].reshape(s["shape"])
    if device is not None: t = t.to(device)
    if dtype  is not None: t = t.to(dtype)
    out[s["name"]] = t
    off += n
  return out

def map_named_to_pid(named_tensors, model):

  return {id(p): named_tensors[name].to(p.device).type_as(p)
    for name, p in model.head.named_parameters()}
def map_named_to_pid(named_tensors, model):

    return {id(p): named_tensors[name].to(p.device).type_as(p)
            for name, p in model.head.named_parameters()}

def clients_round(n_clients,initial_model_name,selection_percentage,spliting_method,client_data,raw_model,class_combs,local_steps,sparcity,threshold=0.01):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  initial_model_name= get_lastgen_model_name(seed_model_name=initial_model_name)
  global_log_df = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
  selected_clients = get_random_clients(n_clients,initial_model_name,selection_percentage)

  log_buffer = global_log_df[global_log_df["model_name"] == initial_model_name]
  r = log_buffer["round_number"].values[0]
  path_to_global_model = log_buffer["path"].values[0]
  path_to_subsets = log_buffer["path_to_subsets"].values[0]
  path_to_class_combs = log_buffer["path_to_class_combs"].values[0]

  if not spliting_method == "i.i.d. sharing":
    path_to_class_combs = " "

  log_file = "/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv"


  dino_model = torch.hub.load('facebookresearch/dino:main', backbone)
  initial_model = DinoClassifier(dino_model=dino_model, num_classes=100, device=device)

  spec = []
  for name, p in initial_model.head.named_parameters():
    spec.append({
        "name": name,
        "shape": tuple(p.shape),
        "numel": p.numel()
    })


  flat_rows = []
  clients_lst = []

  for client_num in selected_clients:
    if os.path.exists("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv"):
      all_clients_df = pd.read_csv(log_file)
      filtered_clients_df = all_clients_df[all_clients_df["initial_model_name"] == initial_model_name]
      if client_num in filtered_clients_df["client_id"].values:
        print(f"Client {client_num} is already trained")
        continue

    client = Client(id=client_num,
                  data=client_data[client_num] ,
                  spliting_method=spliting_method,
                  classes= "all" if spliting_method == "i.i.d. sharing" else class_combs[client_num].tolist(),
                  target_sparsity= sparcity,
                  GM_rounds=   fisher_information_matrix_colibration_rounds,
                  num_local_steps= local_steps,
                  n_clients=n_clients,
                  raw_model = copy.deepcopy(raw_model),
                  path_to_model = path_to_global_model ,
                  path_to_subsets=path_to_subsets,
                  path_to_class_combs=path_to_class_combs
                  )
    clients_lst.append(client)
    flat_rows.append(flatten_from_pid_dict_using_names(client.fisher_score_matrix, client.model, spec))


  stacked = torch.stack(flat_rows)
  var_vec = stacked.var(dim=0, unbiased=True)
  var_by_name = unflatten_to_named(var_vec, spec, device=var_vec.device, dtype=var_vec.dtype)
  all_var = torch.cat([t.reshape(-1) for t in var_by_name.values()])
  mu, sigma = all_var.mean(), all_var.std()
  thresholed = mu + 2 * sigma

  global_grad_mask = {name: (t <= thresholed).float() for name, t in var_by_name.items()}

  print("Global gradient mask is generated")
  for client in clients_lst:
    print( f"{np.where(selected_clients == client_num)[0] +1 }/{len(selected_clients)} ","#"*100)

    client.set_final_grad_mask(map_named_to_pid(global_grad_mask,client.model) )

    client.SGDM_train()
    # client.train_default()
    client.evaluate()
    # Use the save_client method from the Client class
    log = client.create_log(
        model_name=next_id(log_file), # Generate a new model name
        path=f"/content/drive/MyDrive/MLDL_FederatedLearning/models/clients/{next_id(log_file)}.pth", # Generate a new path
        round_number=initial_model_round_num + 1
        )
    client.confirm_save(log['path'][0]) # Save the model

    if not os.path.exists(log_file):
      log.to_csv(log_file, index=False)

      print("new csv file ")
      print(f"name: {log['model_name'][0]} ")
      print(f"path: {log['path'][0]} ")
      print(f"Logged client {client_num} to {log_file}")

    else: # HERE
    # Create a new CSV file IF path doesn't exist
      # This check is no longer necessary as we generate a new id and path every time
      path_check = pd.read_csv(log_file)['model_name'].values # model_name

      # if log['model_name'][0] not in path_check:
      client.confirm_save(log['path'][0])
      log.to_csv(log_file, mode='a', header=False, index=False)
      # print(f"name: {log['model_name'][0]} ")
      # print(f"path: {log['path'][0]} ")
      # print(f"Logged client {client_num} to {log_file}")
      # else :
      #   print("Existing Log")


In [27]:
#@title Random clients selection



selection_percentage = 10 #@param {"type":"integer"}

def get_random_clients(n_clients, initial_model_name, selection_percentage=10):
  global_log = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
  filter = (global_log["model_name"] == initial_model_name)
  filtered_models = global_log[filter]

  r_num =  filtered_models["round_number"].values[0]


  set_seed(int(r_num),is_seed_fixed)
  if os.path.exists("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv"):
    clients_df = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv")
    clients_df = clients_df[clients_df['initial_model_name']== initial_model_name]
    selected_clients = clients_df['client_id'].values
  else:
    selected_clients = np.array([], dtype=np.int16)
  while len(selected_clients) < (selection_percentage / 100 ) * n_clients:
    rand_int = torch.randint(0,n_clients,(1,))[0].item()
    if rand_int not in selected_clients:
      selected_clients = np.append(selected_clients,rand_int)

  return selected_clients

# selected_clients = get_random_clients(n_clients,initial_model_name,selection_percentage)
# print(selected_clients)

In [28]:
# @title Collect clients

def get_clients(initial_model_name):
  clients_data = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv")

  filter =  clients_data['initial_model_name']== initial_model_name
  filtered_clients_data = clients_data[filter] # Using filter to collect clients with specified initial model

  print("Number of all trained clients:", len(clients_data))
  print("Number of clients after filtering:", len(filtered_clients_data))
  return filtered_clients_data


# server circle

In [29]:
# @title generation of new model's name
def global_model_name_path_generator():

  model_name = next_id("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")

  path = "/content/drive/MyDrive/MLDL_FederatedLearning/models/global/" + model_name + ".pth"

  return model_name, path


In [30]:
# @title This function will evaluate the model.</br> The outputs are loss and accuracy
def evaluation(model, data_loader):
  criterion = nn.CrossEntropyLoss()
  model.eval()
  correct = 0
  total = 0
  test_loss = 0

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  with torch.no_grad():
    for images, labels in  data_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)

      _, prediction = torch.max(outputs.data,1)
      loss = criterion(outputs, labels)
      test_loss += loss.item() * labels.size(0)

      total += labels.size(0)
      correct += (prediction == labels).sum().item()
    accuracy = 100 * correct / total
    loss = test_loss / total
    return accuracy, loss



In [31]:
# @title `get_model` function retrieves and loads the models of filtered clients
def get_model(paths,sample_sizes, backbone, raw_model):
  for index in range(len(paths)):
    state_dict = torch.load(paths.iloc[index]) # load state dict regarding the client number 'index'
    raw_model.head.load_state_dict(state_dict) # set the state dict based on client
    raw_model.to(device)
    yield (raw_model,sample_sizes.iloc[index]) # this command throws model one at the time so less time and resouces will be used





In [32]:
#@title <h2>FedAvg</h2>
def fed_avg(df, raw_model):
  total_samples = df["client_train_size"].sum() # Calculate the total number of samples of clients wich had contributed
  global_head = None # This variable stores the weights we want to modify

  models = get_model(df["path"],df["client_train_size"], df.iloc[0]["backbone"],raw_model)
  with torch.no_grad():

    for model, sample_size in models:
      if global_head is None:
        global_head = copy.deepcopy(model.head.state_dict())
        global_model = copy.deepcopy(model)
        for k in global_head.keys():
          global_head[k].zero_() # This command sets the tensor to zero

      for k in global_head.keys():
        global_head[k] += model.head.state_dict()[k] * (sample_size / total_samples) # Each weight will be assgin by average of all clients weights

    global_model.head.load_state_dict(global_head) # A model with modified head will be assignd

  return global_model



In [33]:
#@title Save and log global model
def save_log_global_model(filtered_clients_data,global_model,test_loader ,Nc,local_steps):
  model_name, path = global_model_name_path_generator()
  test_accracy, test_loss= evaluation(global_model, test_loader)
  log_path = "/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv"

  prev_global_model_name = filtered_clients_data["initial_model_name"].values[0]
  classes = filtered_clients_data["classes"].values[-1]
  global_model_log = filtered_clients_data.drop(["client_id","train_loss","classes", "client_train_size","client_test_size","duration",],axis=1)


  global_model_log = global_model_log.iloc[0]






  global_model_log["num_of_participants"] = len(filtered_clients_data)
  global_model_log["prev_global_model_name"] = prev_global_model_name
  global_model_log["model_name"]= model_name
  global_model_log["accuracy"] = test_accracy
  global_model_log["loss"] = test_loss
  global_model_log["time"] = get_current_time()
  global_model_log["path"] = path
  global_model_log["Measurement_criteria"] = "accuracy,loss"
  global_model_log["contributors"] = list(filtered_clients_data["model_name"])
  global_model_log["aggregation_method"] = "FedAvg_globalmasking"
  global_model_log["momentum_vector_path"] = None
  global_model_log["Nc"] = Nc
  global_model_log["num_local_steps"] = local_steps








  global_model_log = pd.DataFrame(global_model_log).T

  global_model_log = global_model_log[
      ['backbone',
      'num_of_clients',
      'splitting_method',
      'Nc',
      'num_local_steps',
      'aggregation_method',
      'Measurement_criteria',
      'accuracy',
      'loss',
      'size_of_dataset',
      'train_test_ratio',
      'round_number',
      'num_of_participants',
      'model_name',
      'prev_global_model_name',
      'contributors',
      'path',
      'momentum_vector_path',
      'path_to_subsets',
      'path_to_class_combs',
      'time']
]



  flag = False
  if os.path.exists(path):
    print("Model already exist")
    flag = True

  if os.path.exists(log_path):
    global_log_df = pd.read_csv(log_path)
    if model_name in global_log_df["model_name"].values:
      print("record already exist")
      flag = True
    if not flag:
      global_model_log.to_csv(log_path, mode='a', header=False, index=False)
  else:
    if not flag:
      global_model_log.to_csv(log_path, index=False, header=True)


  if not flag:
    torch.save(global_model.head.state_dict(),  path )
  return global_model_log


# Main Loop

In [67]:
seed_model_name = "4b94d332-b21c-4e2b-af6f-f7801a35fc4c" #@param{"type":"string"}


dino_model = torch.hub.load('facebookresearch/dino:main', backbone)
raw_model = DinoClassifier(dino_model=dino_model, num_classes=100, device=device)

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


In [68]:

load_data = True #@param{"type":"boolean"}



if load_data:

  initial_model_log_df = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
  path_to_subsets =initial_model_log_df[initial_model_log_df["model_name"] == seed_model_name]["path_to_subsets"].values[0]
  path_to_class_combs =initial_model_log_df[initial_model_log_df["model_name"] == seed_model_name]["path_to_class_combs"].values[0]
  print(path_to_subsets)
  initial_model_name= get_lastgen_model_name(seed_model_name=seed_model_name)
  initial_model_path = initial_model_log_df[initial_model_log_df["model_name"] == initial_model_name]["path"].values[0]
  print(initial_model_name)


  dino_model = torch.hub.load('facebookresearch/dino:main', backbone)
  initial_model = DinoClassifier(dino_model=dino_model, num_classes=100, device=device)
  initial_model.head.load_state_dict(torch.load(initial_model_path))

  client_data = torch.load(path_to_subsets, weights_only=False)
  if not spliting_method == "i.i.d. sharing":
    class_combs = torch.load(path_to_class_combs)



/content/drive/MyDrive/MLDL_FederatedLearning/client_subsets/client_data_noniid_100clients_6dc17b0a-f776-4036-a63a-a659b4f2dc12.pth
4b94d332-b21c-4e2b-af6f-f7801a35fc4c -->
4b94d332-b21c-4e2b-af6f-f7801a35fc4c


Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


In [69]:
# @title Whole FL Circle
from google.colab import output
report_log = None
round_log = ""
start_time = time.perf_counter()
while True:
  start_time_single =time.perf_counter()



  initial_model_name= get_lastgen_model_name(seed_model_name=seed_model_name)
  initial_model_log_df = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
  initial_model_round_num = initial_model_log_df[initial_model_log_df["model_name"] == initial_model_name]["round_number"].values[0]

  if initial_model_round_num > 51:

    break

  print(f"Seed Model name: {seed_model_name} , Initial model name: {initial_model_name} ,  Round number: {initial_model_round_num}")



  clients_round(n_clients,initial_model_name,selection_percentage,spliting_method,client_data,raw_model,class_combs, local_steps,sparcity)
  filtered_clients_data = get_clients(initial_model_name)

  if len(filtered_clients_data) >= n_clients / selection_percentage :
    global_model = fed_avg(filtered_clients_data,raw_model)
    report_log = save_log_global_model(filtered_clients_data,global_model,test_loader,Nc,local_steps)
  output.clear()
  round_log += f"round {initial_model_round_num} / {int(100 / (local_steps/4))} >> loss: {report_log['loss'].values[0]}, accuracy: {report_log['accuracy'].values[0]},Single round timer:{ round((time.perf_counter() - start_time_single), 1) }s , Timer:{ round((time.perf_counter() - start_time)/60, 1) }m , model_name:{initial_model_name}\n"
  print(round_log)




round 0 / 50 >> loss: 6.776195863342285, accuracy: 1.47,Single round timer:60.9s , Timer:1.0m , model_name:4b94d332-b21c-4e2b-af6f-f7801a35fc4c
round 1 / 50 >> loss: 6.270820002746582, accuracy: 1.73,Single round timer:57.1s , Timer:2.0m , model_name:73391a7e-4c11-42b1-82a5-5b6c282e9f99
round 2 / 50 >> loss: 5.884026219177246, accuracy: 2.94,Single round timer:61.0s , Timer:3.0m , model_name:20f1670b-8042-4728-9868-fc0a3692c42c
round 3 / 50 >> loss: 5.628341864013672, accuracy: 3.89,Single round timer:59.7s , Timer:4.0m , model_name:7ddb0096-b5c3-4ef7-9db2-3406d060ddae
round 4 / 50 >> loss: 5.391633323669433, accuracy: 4.08,Single round timer:60.2s , Timer:5.0m , model_name:2f30ff22-c0a0-46af-964a-eea5ccb11234
round 5 / 50 >> loss: 5.204597003936768, accuracy: 6.02,Single round timer:60.7s , Timer:6.0m , model_name:54c39670-0791-4205-b39b-ebaf751395ce
round 6 / 50 >> loss: 5.04687048034668, accuracy: 7.09,Single round timer:60.6s , Timer:7.0m , model_name:584d2f35-cc77-4195-9572-b78df0